# Question3

## 3.1 - Training

### 3.1.1 - Resize Images

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

images = []

# TODO: check if grayscale is correct for HoG

# Read in each image, convert to Gray and resize, then append to list
for i in range(1,15):
    img_temp = cv2.imread("./car"+str(i)+".jpg")
    img_temp = cv2.cvtColor(img_temp, cv2.COLOR_BGR2GRAY) 
    img_temp = cv2.resize(img_temp, (128,128))
    images.append(img_temp)

### 3.1.2 - Compute HoG features

In [4]:
# Make a function which takes list of images as arguments
# and delivers list of HoG features as output. 

def hog_features(imgs_arr, cell_size=(4,4), block_size=(2,2), nbins=9):
    hog_feats_arr = []
    count = 0
    for img in imgs_arr:
        # Check window size
        if(img.shape[0]%16!=0 or img.shape[1]%16!=0):
            raise Exception('Invalid Image Size')
        else:
            # Compute window size
            win_XY = img.shape[0] // cell_size[0] * cell_size[1]
            
            # Compute blocks
            block_XY = block_size[0] * cell_size[0]
            
            # Create HoG object
            hog = cv2.HOGDescriptor(_winSize = (win_XY, win_XY),
                                    _blockSize = (block_XY, block_XY),
                                    _blockStride = (cell_size[1], cell_size[0]),
                                    _cellSize = (cell_size[1], cell_size[0]),
                                    _nbins = nbins
                                   )
            
            # Compute number of cells 
            n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])
            
            # Compute HoG features
            hog_feats = hog.compute(img) \
                            .reshape(n_cells[1] - block_size[1] + 1,
                                     n_cells[0] - block_size[0] +1, 
                                     block_size[1], block_size[0], nbins) \
                            .transpose((1, 0, 3, 2, 4))
            
            hog_feats_arr.append(hog_feats)
            
    return hog_feats_arr
            
feats = hog_features(images)

### 3.1.3 - Calculate mean feature map across training images

In [45]:
# Map the HoGs
# Do we iterate over each HoG and take the average??

# Should still have 4 blocks

height, width = feats[0][:,:,0,0,0].shape
blocks_avg = []

# Iterate over each block (0,0), (0,1), (1,0) and (1,1)
# Iterate over each pixel, and find the average of that pixel for
# the 14 trained image
# Final array is a nested array with the average pixel for each block
for l in range(2):
    for k in range(2):
        block_avg = np.zeros((height, width))
        for h in range(height):
            pixel_sum = 0
            for w in range(width):
                for i in range(len(feats)):
                    pixel_sum = feats[i][:,:,k,l,0][h][w]
                pixel_avg = pixel_sum / (len(feats))
                block_avg[w][h]= pixel_avg
                
        blocks_avg.append(block_avg)

### 3.1.4 - Repeat steps 3.1.2 - 3.1.3 for images flipped vertically

In [46]:
imgs_flip = []

# Flip image about the vertical axis -> ie across y-axis
for img in images:
    imgs_flip.append(cv2.flip(img, 1))
    
feats_flipped = hog_features(imgs_flip)

# TODO: CHECK calculation for mean feature map

height, width = feats[0][:,:,0,0,0].shape
blocks_avg_rotated = []

# Iterate over each block (0,0), (0,1), (1,0) and (1,1)
# Iterate over each pixel, and find the average of that pixel for
# the 14 trained image
# Final array is a nested array with the average pixel for each block
for l in range(2):
    for k in range(2):
        block_avg = np.zeros((height, width))
        for h in range(height):
            pixel_sum = 0
            for w in range(width):
                for i in range(len(feats_flipped)):
                    pixel_sum = feats_flipped[i][:,:,k,l,0][h][w]
                pixel_avg = pixel_sum / (len(feats))
                block_avg[w][h]= pixel_avg
                
        blocks_avg_rotated.append(block_avg)

### 3.1.5 - Display 9 orientation channels for first block